# TOOL FOR CALCULTING PUT/CALL OPTIONS VALUE IN THE BINOMIAL PRICING MODEL
***

Here we price the option using the simple parametrization in which $S_{i+1} = uS$ with probability $p$ and $vS$ with probability $(1-p)$. The option price is
$$V_i = \frac{V^+ _{i+1} - V^- _{i+1}}{u-V} + \frac{1}{e^{r\delta t}}\frac{uV^- _{i+1} - vV^+ _{i+1}}{u-V} $$
where $V_{i+1} ^{\pm}$ are the option values at the next step. Steps are defined by $\delta t = T/N$, $i=0...N$.

In [22]:
import numpy as np
from typing import List, Dict

class OptionTypeInvalid(Exception):
    def __init__(self,  message="Option type invalid"):
        self.message = message
        super().__init__(self.message)

class OptionSpeciesInvalid(Exception):
    def __init__(self,  message="Option species invalid"):
        self.message = message
        super().__init__(self.message)

def option_payoff(option_type: 'str', asset_price: 'float', strike_price: 'float') -> float:
    """ Returns the value of this option give an asset price and strike price

    Attributes
    ----------
    option_type: str
        Either 'call' or 'put'
    asset_price: float
    strike_price: float

    Returns
    -------
    float
        The value of the option
    """
    if option_type == 'call':
        return max(0,asset_price-strike_price)
    elif option_type == 'put':
        return max(0, strike_price-asset_price)
    else:
        raise OptionTypeInvalid()
    

def option_prices(option_type: 'str', option_species : 'str', strike_price: 'float', stock_init_price: 'float', u: 'float', v: 'float', risk_free_rate: 'float' , tot_time: 'float', n_steps: 'int') -> List[Dict[float, float]]:
    """ Returns the values of the option given the option details, the risk free rate, the maturity and the number of steps in the simulation

    Attributes
    ----------
    option_type: str
        Either 'call' or 'put'
    option_species: str
        Either 'european' (default) or 'american'
    strike_price: float
    stock_init_price: float
    u: float
    v: float
    risk_free_rate: float
    tot_time: float
    n_steps: int

    Returns
    -------
    List[Dict[str, int]]
        The values of the stock, of the option payoff and of the option at time step i
    """
    step_size =  tot_time/n_steps
    n_steps += 1
    stock_prices = {i: [None for _ in range(i+1)] for i in range(n_steps)}
    option_payoffs = {i: [None for _ in range(i+1)] for i in range(n_steps)}
    option_values = {i: [None for _ in range(i+1)] for i in range(n_steps)}
    for i in range(n_steps):
        for j in range(i+1):
            stock_prices[i][j] = stock_init_price * (u**(i-j)) * (v**(j))
            option_payoffs[i][j] = option_payoff(option_type, stock_prices[i][j],strike_price)
    option_values[n_steps-1] = option_payoffs[n_steps-1]
    for i in range(n_steps):
        for j in range(n_steps-i-1):
            i_retro = n_steps-i-1
            if option_species == 'european':
                option_values[i_retro-1][j] = (option_values[i_retro][j]-option_values[i_retro][j+1])/(u-v) + (u*option_values[i_retro][j+1]-v*option_values[i_retro][j])/((u-v)*np.exp(risk_free_rate*step_size))
            elif option_species == 'american':
                option_values[i_retro-1][j] = max( (option_values[i_retro][j]-option_values[i_retro][j+1])/(u-v) + (u*option_values[i_retro][j+1]-v*option_values[i_retro][j])/((u-v)*np.exp(risk_free_rate*step_size)),
                                                  option_payoff(option_type, stock_prices[i_retro-1][j], strike_price))
            else:
                raise OptionSpeciesInvalid()
            
    return [stock_prices, option_payoffs,option_values]



Example of calculation

In [ ]:
u = 1.0604
v= 0.9431
r = 0.1 # annual interest
t_tot = 1/3 # in years, so 4 months
n_steps = 4 # 4 steps aka 4 months ahead
stck_prices, opt_payoffs, opt_vals = option_prices('call', 'european', 100, 100, u,v, r, 1/3, )

In [2]:
stck_prices

{0: [100.0],
 1: [106.04, 94.31],
 2: [112.444816, 100.006324, 88.94376100000001],
 3: [119.23648288640001,
  106.04670596960001,
  94.31596416440001,
  83.88286099910002],
 4: [126.43836645273856,
  112.45192701016386,
  100.01264839992977,
  88.94938580344567,
  79.10992620825122]}

In [3]:
opt_payoffs

{0: [0],
 1: [6.040000000000006, 0],
 2: [12.444816000000003, 0.006324000000006436, 0],
 3: [19.23648288640001, 6.046705969600012, 0, 0],
 4: [26.438366452738563, 12.451927010163857, 0.01264839992977329, 0, 0]}

In [4]:
opt_vals

{0: [6.1369300009326295],
 1: [9.449690650117873, 2.0972146854304263],
 2: [14.097670617838276, 3.7975870074382243, 0.003851269930354978],
 3: [20.06635362251241, 6.876576705712424, 0.006979427076532846, 0.0],
 4: [26.438366452738563, 12.451927010163857, 0.01264839992977329, 0, 0]}